## 1. 라이브러리 로드

In [ ]:
!pip install timm

In [ ]:
pip install pandas scikit-learn matplotlib seaborn 

In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (필요시)
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

# 시각화 스타일
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 2. 데이터 로드

In [ ]:
# 데이터 경로 설정
BASE_DIR = '/home/realtheai/cv_competetion'
TRAIN_CSV = f"{BASE_DIR}/data/train.csv"
TEST_CSV = f"{BASE_DIR}/data/sample_submission.csv"
TRAIN_IMG_DIR = f"{BASE_DIR}/data/train"
TEST_IMG_DIR = f"{BASE_DIR}/data/test"
META_CSV = f"{BASE_DIR}/data/meta.csv"

# CSV 파일 로드
train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)
meta_df = pd.read_csv(META_CSV)

print(f"Train 데이터: {len(train_df)}개")
print(f"Test 데이터: {len(test_df)}개")
print(f"클래스 수: {len(meta_df)}개")
print(f"\nTrain/Test 비율: 1:{len(test_df)/len(train_df):.1f}")

In [ ]:
# 데이터 미리보기
print("\n=== Train Data ===")
display(train_df.head())

print("\n=== Meta Data (클래스 정보) ===")
display(meta_df)

## 3. 클래스 분포 분석

In [ ]:
# 클래스별 개수
class_counts = train_df['target'].value_counts().sort_index()

# 클래스 이름 매핑
class_names = dict(zip(meta_df['target'], meta_df['class_name']))

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 막대 그래프
axes[0].bar(class_counts.index, class_counts.values, color='steelblue', alpha=0.8)
axes[0].set_xlabel('Class ID', fontsize=12)
axes[0].set_ylabel('Count', fontsize=12)
axes[0].set_title('Train Data - Class Distribution', fontsize=14, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# 파이 차트
axes[1].pie(class_counts.values, labels=[f'{i}\n({class_names.get(i, "Unknown")[:15]})' for i in class_counts.index], 
            autopct='%1.1f%%', startangle=90)
axes[1].set_title('Class Distribution (Pie Chart)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('/home/realtheai/cv_competetion/eda_class_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n=== 클래스별 샘플 수 ===")
for class_id, count in class_counts.items():
    print(f"Class {class_id:2d} ({class_names.get(class_id, 'Unknown'):20s}): {count:3d}개")

In [ ]:
# 불균형 정도 확인
max_count = class_counts.max()
min_count = class_counts.min()
imbalance_ratio = max_count / min_count

print(f"\n⚠️ 클래스 불균형 정도")
print(f"최대 클래스: {max_count}개")
print(f"최소 클래스: {min_count}개")
print(f"불균형 비율: {imbalance_ratio:.2f}:1")

if imbalance_ratio > 3:
    print("\n💡 제안: 클래스 불균형이 심합니다. Weighted Loss 또는 Oversampling 고려하세요!")

## 4. 이미지 크기 분석

In [ ]:
# 샘플 이미지들의 크기 수집 
print("이미지 크기 분석 중... (샘플 100개)")

image_sizes = []
aspect_ratios = []

for img_name in train_df['ID'].sample(min(100, len(train_df))):  # 샘플 100개만
    img_path = os.path.join(TRAIN_IMG_DIR, img_name)
    if os.path.exists(img_path):
        img = Image.open(img_path)
        w, h = img.size
        image_sizes.append((w, h))
        aspect_ratios.append(w / h)

widths = [s[0] for s in image_sizes]
heights = [s[1] for s in image_sizes]

# 시각화
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Width 분포
axes[0].hist(widths, bins=20, color='skyblue', edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Width (pixels)', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Image Width Distribution', fontsize=13, fontweight='bold')
axes[0].axvline(np.mean(widths), color='red', linestyle='--', label=f'Mean: {np.mean(widths):.0f}')
axes[0].legend()

# Height 분포
axes[1].hist(heights, bins=20, color='lightcoral', edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Height (pixels)', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].set_title('Image Height Distribution', fontsize=13, fontweight='bold')
axes[1].axvline(np.mean(heights), color='red', linestyle='--', label=f'Mean: {np.mean(heights):.0f}')
axes[1].legend()

# Aspect Ratio 분포
axes[2].hist(aspect_ratios, bins=20, color='lightgreen', edgecolor='black', alpha=0.7)
axes[2].set_xlabel('Aspect Ratio (W/H)', fontsize=12)
axes[2].set_ylabel('Frequency', fontsize=12)
axes[2].set_title('Aspect Ratio Distribution', fontsize=13, fontweight='bold')
axes[2].axvline(np.mean(aspect_ratios), color='red', linestyle='--', label=f'Mean: {np.mean(aspect_ratios):.2f}')
axes[2].legend()

plt.tight_layout()
plt.savefig('/home/realtheai/cv_competetion/eda_image_sizes.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n평균 이미지 크기: {np.mean(widths):.0f} x {np.mean(heights):.0f}")
print(f"평균 Aspect Ratio: {np.mean(aspect_ratios):.2f}")

## 5. 클래스별 샘플 이미지 시각화

In [ ]:
# 각 클래스별로 샘플 이미지 1개씩 출력
num_classes = len(meta_df)
cols = 6
rows = (num_classes + cols - 1) // cols

fig, axes = plt.subplots(rows, cols, figsize=(18, rows*3))
axes = axes.flatten()

for idx, (class_id, class_name) in enumerate(zip(meta_df['target'], meta_df['class_name'])):
    # 해당 클래스의 첫 번째 이미지 찾기
    sample_row = train_df[train_df['target'] == class_id].iloc[0]
    img_path = os.path.join(TRAIN_IMG_DIR, sample_row['ID'])
    
    if os.path.exists(img_path):
        img = Image.open(img_path)
        axes[idx].imshow(img, cmap='gray')
        axes[idx].set_title(f'Class {class_id}: {class_name}', fontsize=10, fontweight='bold')
        axes[idx].axis('off')
    else:
        axes[idx].text(0.5, 0.5, 'Image not found', ha='center', va='center')
        axes[idx].axis('off')

# 빈 subplot 숨기기
for idx in range(num_classes, len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.savefig('/home/realtheai/cv_competetion/eda_sample_images_by_class.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Train vs Test 데이터 비교 (노이즈 분석)

In [ ]:
# Train과 Test 샘플 비교 (각 3개씩)
print("💡 Train vs Test 이미지 비교 - 노이즈 패턴을 육안으로 확인하세요!")

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Train 샘플 3개
for i in range(3):
    img_name = train_df['ID'].iloc[i]
    img_path = os.path.join(TRAIN_IMG_DIR, img_name)
    if os.path.exists(img_path):
        img = Image.open(img_path)
        axes[0, i].imshow(img, cmap='gray')
        axes[0, i].set_title(f'TRAIN Sample {i+1}', fontsize=12, fontweight='bold')
        axes[0, i].axis('off')

# Test 샘플 3개
for i in range(3):
    img_name = test_df['ID'].iloc[i]
    img_path = os.path.join(TEST_IMG_DIR, img_name)
    if os.path.exists(img_path):
        img = Image.open(img_path)
        axes[1, i].imshow(img, cmap='gray')
        axes[1, i].set_title(f'TEST Sample {i+1}\n(노이즈 확인!)', fontsize=12, fontweight='bold', color='red')
        axes[1, i].axis('off')

plt.tight_layout()
plt.savefig('/home/realtheai/cv_competetion/eda_train_vs_test_samples.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ 확인사항:")
print("1. Test 이미지에 구겨짐, 물얼룩, 빛번짐 등의 노이즈가 있나요?")
print("2. Train 대비 Test의 화질이나 밝기가 다른가요?")
print("3. 회전, 기울어짐 등이 있나요?")
print("\n이 정보를 바탕으로 Augmentation 전략을 수립하세요!")

## 7. 이미지 밝기 분석

In [ ]:
# Train과 Test의 평균 픽셀 값 비교 (밝기)
print("이미지 밝기 분석 중... (샘플 50개)")

train_brightness = []
test_brightness = []

for img_name in train_df['ID'].sample(min(50, len(train_df))):
    img_path = os.path.join(TRAIN_IMG_DIR, img_name)
    if os.path.exists(img_path):
        img = np.array(Image.open(img_path).convert('L'))  # Grayscale
        train_brightness.append(img.mean())

for img_name in test_df['ID'].sample(min(50, len(test_df))):
    img_path = os.path.join(TEST_IMG_DIR, img_name)
    if os.path.exists(img_path):
        img = np.array(Image.open(img_path).convert('L'))
        test_brightness.append(img.mean())

# 시각화
plt.figure(figsize=(10, 6))
plt.hist(train_brightness, bins=20, alpha=0.6, label='Train', color='blue')
plt.hist(test_brightness, bins=20, alpha=0.6, label='Test', color='red')
plt.xlabel('Average Pixel Value (Brightness)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Brightness Distribution: Train vs Test', fontsize=14, fontweight='bold')
plt.legend(fontsize=12)
plt.grid(alpha=0.3)
plt.savefig('/home/realtheai/cv_competetion/eda_brightness_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nTrain 평균 밝기: {np.mean(train_brightness):.2f}")
print(f"Test 평균 밝기: {np.mean(test_brightness):.2f}")
print(f"밝기 차이: {abs(np.mean(train_brightness) - np.mean(test_brightness)):.2f}")

if abs(np.mean(train_brightness) - np.mean(test_brightness)) > 10:
    print("\n💡 제안: Train과 Test의 밝기 차이가 큽니다. Brightness Augmentation 추가를 고려하세요!")

## 8. EDA 결과 요약 및 Augmentation 전략 제안

In [ ]:
print("="*80)
print("📊 EDA 결과 요약")
print("="*80)

print(f"\n1. 데이터 구성")
print(f"   - Train: {len(train_df)}개")
print(f"   - Test: {len(test_df)}개 (Train의 {len(test_df)/len(train_df):.1f}배)")
print(f"   - 클래스 수: {num_classes}개")

print(f"\n2. 클래스 불균형")
print(f"   - 최대/최소 비율: {imbalance_ratio:.2f}:1")
if imbalance_ratio > 3:
    print(f"   ⚠️ 불균형 심함 → Weighted Loss 고려 필요")

print(f"\n3. 이미지 크기")
print(f"   - 평균 크기: {np.mean(widths):.0f} x {np.mean(heights):.0f}")
print(f"   - 평균 Aspect Ratio: {np.mean(aspect_ratios):.2f}")

print(f"\n4. Train vs Test 밝기")
print(f"   - Train: {np.mean(train_brightness):.2f}")
print(f"   - Test: {np.mean(test_brightness):.2f}")
print(f"   - 차이: {abs(np.mean(train_brightness) - np.mean(test_brightness)):.2f}")


## 9. 생성된 이미지 파일 확인

In [ ]:
print("\n📁 생성된 EDA 결과 이미지:")
import os
base_dir = '/home/realtheai/cv_competetion'
eda_images = [
    'eda_class_distribution.png',
    'eda_image_sizes.png',
    'eda_sample_images_by_class.png',
    'eda_train_vs_test_samples.png',
    'eda_brightness_comparison.png'
]

for img_file in eda_images:
    img_path = os.path.join(base_dir, img_file)
    if os.path.exists(img_path):
        print(f"   ✅ {img_path}")
    else:
        print(f"   ❌ {img_path} (생성 실패)")
